In [1]:
import qiskit
import numpy as np

An arbitrary unitary matrix on an n-qubit system may be written as a product of at most $2^{n-1}(2^n -1)$ two-level uinitary matrices

In [2]:
def decomp(matrix):
  """Follows procedure in N&C 4.5.1"""
  if len(matrix) <= 2:
    return matrix

  #the list of matrices whose product is the input matrix
  #U3 U2 U1 U = I
  # U = dag(U1) dag(U2) dag(U3)
  unitary_list = []
  offset = 0
  temp = matrix

  #(4.44) - (4.49)
  while(offset <= len(matrix) - 2):
  
    unitary_list.append(np.eye(len(matrix), dtype=complex))
    if offset != 0:
      unitary_list[offset][0][0] = np.conj(temp[0][0])

    if temp[1+offset][0] != 0:
      unitary_list[offset][0][0] = np.conj(temp[0][0]) / np.sqrt(np.abs(temp[0][0])**2 + np.abs(temp[1+offset][0])**2)
      unitary_list[offset][0][1+offset] = np.conj(temp[1+offset][0]) / np.sqrt(np.abs(temp[0][0])**2 + np.abs(temp[1+offset][0])**2)
      unitary_list[offset][1+offset][0] = temp[1+offset][0] / np.sqrt(np.abs(temp[0][0])**2 + np.abs(temp[1+offset][0])**2)
      unitary_list[offset][1+offset][1+offset] = -temp[0][0] / np.sqrt(np.abs(temp[0][0])**2 + np.abs(temp[1+offset][0])**2)

    temp = np.matmul(unitary_list[offset], temp)
    offset += 1
  
  #finally (4.50)
  temp2 = np.eye(len(matrix), dtype=complex)
  temp2[-2][-2] = np.conj(temp[-2][-2])
  temp2[-1][-1] = np.conj(temp[-1][-1])
  temp2[-2][-1] = np.conj(temp[-1][-2])
  temp2[-1][-2] = np.conj(temp[-2][-1])
  unitary_list.append(temp2)

  return unitary_list
    

In [3]:
unitary_3 = .5 * np.array([[1,-1j, -1+1j], [1j, 1, 1+1j], [1+1j, -1+1j, 0]])
d3_decomp = decomp(unitary_3)
#use isclose because some values are of order 1e-16
#U3 U2 U1 U = I
assert np.isclose(np.matmul(d3_decomp[2], np.matmul(d3_decomp[1], np.matmul(d3_decomp[0], unitary_3))), np.eye(3)).all()

In [4]:
ft_unitary  = .5 * np.array([[1,1,1,1], [1,1j,-1,-1j], [1,-1,1,-1], [1,-1j, -1, 1j]])
ft_decomp = decomp(ft_unitary)
 # U = dag(U1) dag(U2) dag(U3)
 #TODO: debug
from functools import reduce
a = reduce(np.matmul, [np.matrix(ft_decomp[0]).getH(), np.matrix(ft_decomp[1]).getH(), np.matrix(ft_decomp[2]).getH(),np.matrix(ft_decomp[3]).getH()])

In [5]:
def gray_code(bit_string, target_string):
    """
    gray_code("000", "111")
    ['000', '001', '011', '111']
    """
    temp = list(bit_string)
    gray_code_list = [bit_string]
    while temp != list(target_string):
        for index, bit in reversed(list(enumerate(temp))):
            if bit != target_string[index]:
                temp[index] = '0' if bit=='1' else '1'
                gray_code_list.append("".join(temp))
                break
    return gray_code_list

def dif(a, b):
    return [i for i in range(len(a)) if a[i] != b[i]]


In [6]:
from math import log2
def controlled_2unitary(two_level_unitary):
    """Follows procedure from 4.5.2"""

    #0. convert full unitary to a 2x2
    unitary_reduced = np.eye(2, dtype=complex)
    non_trivial_indices = np.flatnonzero(np.diagonal(two_level_unitary)-1)

    #add default values in case does not act nontrivally in 2 places
    #TODO: test this
    if len(non_trivial_indices) == 0:
        non_trivial_indices.append(0)
    if len(non_trivial_indices) == 1:
        non_trivial_indices.append(0)

    unitary_reduced[0][0] = two_level_unitary[non_trivial_indices[0]][non_trivial_indices[0]]
    unitary_reduced[0][1] = two_level_unitary[non_trivial_indices[0]][non_trivial_indices[1]]
    unitary_reduced[1][0] = two_level_unitary[non_trivial_indices[1]][non_trivial_indices[0]]
    unitary_reduced[1][1] = two_level_unitary[non_trivial_indices[1]][non_trivial_indices[1]]
    
    #1. find where unitary acts non-trivially
    a = np.diagonal(two_level_unitary)
    np.flatnonzero(a-1)
    str_list = []
    for el in np.flatnonzero(a-1):
        str_list.append(format(el, f'0{int(log2(len(two_level_unitary)))}b'))

    gray_code_list = gray_code(str_list[0], str_list[1])

    #2.use gates to shuffle the states (Fig. 4.16)  
    circuit = qiskit.QuantumCircuit(int(log2(len(two_level_unitary))))
    #the entries in gray_code_list inform where to place MCNOTs
    for code1, code2 in zip(gray_code_list[:-2], gray_code_list[1:-1]):

        #target is the difference between code1, code2
        target = dif(code1, code2)
        assert len(target) == 1
        target = target[0]

        control_list = []
        for index, bit in enumerate(code1):
            if index != target:
                if bit == '0':
                    circuit.x(index)
                control_list.append(index)
        
        circuit.mcx(control_qubits=control_list, target_qubit= target)
    
    #3. apply controlled unitary, then unshuffle states
    #control bits come from last state in gray_code_list
    target = dif(gray_code_list[-2], gray_code_list[-1])
    assert len(target) == 1
    target = target[0]

    control_list = []
    for index, bit in enumerate(gray_code_list[-1]):
        if index != target:
            if bit == '0':
                circuit.x(index)
            control_list.append(index)

    
    #create unitary using the 2x2 form of the matrix
    unitary_gate = qiskit.extensions.UnitaryGate(unitary_reduced)
    c_unitary_gate = unitary_gate.control(len(control_list))
    # c_unitary_gate = unitary_gate.control(control_list)
    circuit.append(c_unitary_gate, control_list +[target])

    #4. unshuffle states (repeat step 2)
    for code1, code2 in zip(gray_code_list[:-2], gray_code_list[1:-1]):

        #target is the difference between code1, code2
        target = dif(code1, code2)
        assert len(target) == 1
        target = target[0]

        control_list = []
        for index, bit in enumerate(code1):
            if index != target:
                if bit == '0':
                    circuit.x(index)
                control_list.append(index)
        
        circuit.mcx(control_qubits=control_list, target_qubit= target)

    return circuit

In [7]:
example = np.eye(8, dtype=complex)
example[2][2] = .5*(1+1j)
example[2][7] = .5*(1-1j)
example[7][2] = .5*(1-1j)
example[7][7] = .5*(1+1j)
c = controlled_2unitary(example)
print(c)

     ┌───┐     ┌─────────┐┌───┐     
q_0: ┤ X ├──■──┤ Unitary ├┤ X ├──■──
     └───┘  │  └────┬────┘└───┘  │  
q_1: ───────■───────■────────────■──
          ┌─┴─┐     │          ┌─┴─┐
q_2: ─────┤ X ├─────■──────────┤ X ├
          └───┘                └───┘


In [8]:
example2 = np.eye(8, dtype=complex)
example2[0][0] = .5*(1+1j)
example2[0][7] = .5*(1-1j)
example2[7][0] = .5*(1-1j)
example2[7][7] = .5*(1+1j)
c = controlled_2unitary(example2)
print(c)

     ┌───┐     ┌───┐     ┌─────────┐┌───┐     ┌───┐     
q_0: ┤ X ├──■──┤ X ├──■──┤ Unitary ├┤ X ├──■──┤ X ├──■──
     ├───┤  │  └───┘┌─┴─┐└────┬────┘├───┤  │  └───┘┌─┴─┐
q_1: ┤ X ├──■───────┤ X ├─────■─────┤ X ├──■───────┤ X ├
     └───┘┌─┴─┐     └─┬─┘     │     └───┘┌─┴─┐     └─┬─┘
q_2: ─────┤ X ├───────■───────■──────────┤ X ├───────■──
          └───┘                          └───┘          


In [ ]:
#TODO: debug
for u in decomp(ft_unitary):
    controlled_2unitary(u)